# AI허브 건강관리를 위한 음식 이미지

In [ ]:
import os
root_dir = '../../healthy_food'
base_dir = os.path.join(root_dir, 'data')
split_dir = {split_name:os.path.join(base_dir,split_name)
                for split_name in ['train','valid','test']}

## Modeling

In [ ]:
import tensorflow as tf
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, AveragePooling2D, Input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from tf.keras.optimizers import SGD
from keras.regularizers import l2
import keras.backend as K

In [ ]:
# 이미지의 각 픽셀을 255로 나눠주는 것은 필수!
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# valid, test 데이터는 augmentation 하면 안됨!
val_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale= 1./255)

train_generator = train_datagen.flow_from_directory(
    split_dir['train'],
    target_size = (299, 299),
    batch_size = 32,
    seed = 42
)

val_generator = val_datagen.flow_from_directory(
    split_dir['valid'],
    target_size = (299, 299),
    batch_size = 32,
    shuffle=True,
    seed = 42
)

test_generator = test_datagen.flow_from_directory(
    split_dir['test'],
    target_size = (299, 299),
    batch_size = 32,
    seed = 42
)

Found 93143 images belonging to 422 classes.
Found 11590 images belonging to 422 classes.
Found 11172 images belonging to 422 classes.


In [ ]:
for data_batch, labels_batch in train_generator:
    print('Batch data size:', data_batch.shape)
    print('Batch label size:', labels_batch.shape)
    break
print('Generator length:', len(train_generator))

배치 데이터 크기 :  (32, 299, 299, 3)
배치 레이블 크기 :  (32, 422)


In [ ]:
from datetime import datetime
now = datetime.now().strftime('%y%m%d_%H%M%S')
now

'220521_132937'

In [ ]:
# 모델 저장 경로 설정
from datetime import datetime
now = datetime.now().strftime('%y%m%d_%H%M%S')

model_dir = os.path.join(base_dir, 'model')
save_dir = os.path.join(model_dir, now)

if not(os.path.isdir(model_dir)):
    os.mkdir(model_dir)
os.mkdir(save_dir)

model_path = save_dir + '/{epoch:02d}-{val_acc:.4f}.hdf5'
print(model_path)

/Users/cuz/Downloads/Data/model/2022-5-21-12h/{epoch:02d}-{val_acc:.4f}.hdf5


In [ ]:
K.clear_session()

with K.tf.device('/device:GPU:0'): # gpu 사용
    # create the base pre-trained model
    base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=Input(shape=(299, 299, 3)))

    x = base_model.output
    x = AveragePooling2D(pool_size=(8, 8))(x)
    x = Dropout(.4)(x)
    x = Flatten()(x)
    x = Dropout(.4)(x)
    predictions = Dense(422, kernel_regularizer=l2(.0005), activation='softmax')(x)

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)
    
    opt = SGD(lr=.01, momentum=.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=10) # 조기종료 콜백함수 정의
    cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_acc',
                                    verbose=1, save_best_only=True) # 체크포인트 저장

/Users/cuz/miniforge3/envs/tf/lib/python3.9/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
# lr_scheduler : epoch 증가함에 따라 learning_rate를 작게 해 cosf_function이 최적값 주변에서 맴도는 것을 완화!

def schedule(epoch):
    if epoch < 15:
        return .01
    elif epoch < 28:
        return .002
    else:
        return .0004
lr_scheduler = LearningRateScheduler(schedule)

In [ ]:
history2 = model.fit_generator(train_generator,
                    epochs=30, steps_per_epoch= len(train_generator), 
                    validation_data = val_generator,
                    validation_steps= len(val_generator),
                    use_multiprocessing=True,
                    callbacks=[cb_checkpoint, early_stopping, lr_scheduler]
                    )

/var/folders/zq/5f2h2lwd7vd8py_tdcdkh4lm0000gn/T/ipykernel_7264/496670375.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history2 = model.fit_generator(train_generator,


Epoch 1/30


2022-05-21 12:18:03.356013: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-05-21 12:18:05.434261: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
Exception in thread Thread-39:
Traceback (most recent call last):
  File "/Users/cuz/miniforge3/envs/tf/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/cuz/miniforge3/envs/tf/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/cuz/miniforge3/envs/tf/lib/python3.9/site-packages/keras/utils/data_utils.py", line 745, in _run
    with closing(self.executor_fn(_SHARED_SEQUENCES)) as executor:
  File "/Users/cuz/miniforge3/envs/tf/lib/python3.9/site-packages/keras/utils/data_utils.py", line 722, in pool_fn
    pool = get_pool_class(True)(
  File "/Users/cuz/miniforge3/envs/tf/lib/python3.9/multiprocessing/context.py", line 119